In [95]:
import pandas as pd 
import numpy as np
import datetime as dt #installs necessary packages

path='/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/raw datasets' #sets directory to proper folder

ur20data=pd.read_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/raw datasets/UR2020.csv') #opens 2020 unemployment data file
ur20data['CountyName']=ur20data['County Name/State Abbreviation'].astype(str) #convert county names to strings
ur20data['Period']=ur20data['Period'].astype(str) #converts dates to strings

ur20data=ur20data[~ur20data['CountyName'].str.endswith('PR')] #removes entries from puerto rico
ur20data=ur20data[~ur20data['Period'].str.startswith('19')] #removes dates from 2019

ur20data['Period']=ur20data['Period'].replace(['20-Jan','20-Feb','20-Mar','20-Apr','20-May','20-Jun','20-Jul','20-Aug','20-Sep','20-Oct','20-Nov','Dec-20 p'],['2020-01-31','2020-02-29','2020-03-31','2020-04-30','2020-05-31','2020-06-30','2020-07-31','2020-08-31','2020-09-30','2020-10-31','2020-11-30','2020-12-31']) #changes dates to values reconizable by dt package
ur20data['Date']=pd.to_datetime(ur20data['Period'], errors='coerce')
ur20data['fips']=ur20data['LAUS Code'].astype(str)
ur20data['fips'] = [x[2:-8] for x in ur20data['fips']]

del ur20data['LAUS Code']
del ur20data['statefips']
del ur20data['countyfips']
del ur20data['County Name/State Abbreviation']
del ur20data['labor force']
del ur20data['Employed']
del ur20data['Unemployed']
del ur20data['Period'] #remove unneeded columns

In [96]:
ur21data=pd.read_csv(path+'/UR2021.csv')

ur21data['CountyName']=ur21data['County Name/State Abbreviation'].astype(str) #convert county names to strings
ur21data['Period']=ur21data['Period'].astype(str) #converts dates to strings

ur21data=ur21data[~ur21data['CountyName'].str.endswith('PR')] #removes entries from puerto rico
ur21data=ur21data[~ur21data['Period'].str.endswith('20')] #removes dates from 2020
ur21data['Period']=ur21data['Period'].replace(['Jan-21','Feb-21','Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21 p'],['2021-01-31','2021-02-28','2021-03-31','2021-04-30','2021-05-31','2021-06-30','2021-07-31','2021-08-31','2021-09-30','2021-10-31','2021-11-30','2021-12-31'])

ur21data['Date']=pd.to_datetime(ur21data['Period'], errors='coerce') #convert st5ring to datetime


In [97]:

ur21data['fips']=ur21data['LAUS Code'].astype(str)
ur21data['fips'] = [x[2:-8] for x in ur21data['fips']] #convert fips to string and change 

del ur21data['LAUS Code']
del ur21data['statefips']
del ur21data['countyfips']
del ur21data['County Name/State Abbreviation']
del ur21data['labor orce']
del ur21data['Employed']
del ur21data['Unemployed']
del ur21data['Period'] #remove unneeded columns 

ur20data=ur20data.rename(columns={'unemployment_rate': 'UR_rate'}) #rename to match other df

frames=[ur20data, ur21data]
urdata=pd.concat(frames) #combines 2 UR df's


In [98]:
vaxdata=pd.read_csv(path+'/vaccinations2021.csv') #opens vaccine dataset
#footnotes here: https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh
vaxdata['Date']=vaxdata['Date'].astype(str)
vaxdata=vaxdata[~vaxdata['Date'].str.endswith('2022')]
vaxdata=vaxdata[~vaxdata['Recip_State'].str.startswith('PR')]
vaxdata['Date']=pd.to_datetime(vaxdata['Date'], errors='coerce')
vaxdata=vaxdata.sort_values(['Date'])

vaxdata=vaxdata[~vaxdata['Recip_State'].str.endswith('VI')]
vaxdata=vaxdata[~vaxdata['Recip_County'].str.startswith('Kalawao')]
vaxdata=vaxdata[~vaxdata['Recip_State'].str.endswith('GU')] #remove unneeded values

vaxdata=vaxdata.loc[(vaxdata['Date'] == '12/31/2020') |
                         (vaxdata['Date'] == '01/31/2021') | 
                         (vaxdata['Date'] == '02/28/2021') | (vaxdata['Date']=='03/31/2021') | (vaxdata['Date']=='04/30/2021') | (vaxdata['Date']=='05/31/2021') | (vaxdata['Date']=='06/30/2021') | (vaxdata['Date']=='07/31/2021') | (vaxdata['Date']=='08/31/2021') | (vaxdata['Date']=='09/30/2021') | (vaxdata['Date']=='10/31/2021') | (vaxdata['Date']=='11/30/2021') | (vaxdata['Date']=='12/31/2021')]
#filters df to needed rows
vaxdata=vaxdata[~vaxdata['FIPS'].str.endswith('UNK')] #removes data from unknown muni
vaxdata=vaxdata.rename(columns={'FIPS': 'fips'}) #rename column to match others


/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [99]:
urvaxdata=pd.merge(urdata, vaxdata,  how='left', left_on=['Date','fips'], right_on = ['Date','fips']) #merge UR and vax data

In [100]:
demodata=pd.read_csv(path+'county_demographics/demodata.csv') #open demographics dataset 
demodata['GEO_ID'] = [x[9:] for x in demodata['GEO_ID']]
demodata=demodata.rename(columns={'GEO_ID': 'fips'})
demodata=demodata.filter(['fips','NAME','DP05_0002PE','DP05_0003PE','DP05_0037PE','DP05_0038PE','DP05_0039PE','DP05_0044PE','DP05_0052PE','DP05_0070PE','DP05_0071PE'])
demodata=demodata.rename(columns={'GEO_ID': 'fips'}) #change fips code formatting & rename
demodata.drop([0], axis=0, inplace=True)

demodata=demodata[~demodata['NAME'].str.endswith('Puerto Rico')] 
countynames=demodata['NAME'].str.split(', ', expand=True)
demodata=pd.concat([demodata, countynames], axis=1) #changing formatting of fips and add names of counties 

demodata['State']=demodata[1].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY','District of Columbia':'DC'})

demodata['CountyName']=demodata[0]+', '+demodata['State'] #change formatting of state names


/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,4,6,8,10,12,14,18,20,22,23,24,25,26,27,28,29,30,32,34,36,38,40,42,44,46,48,50,51,52,53,54,55,56,57,58,60,62,63,64,65,66,67,68,69,70,71,74,76,78,79,80,81,82,84,86,87,88,89,90,91,92,93,94,96,98,100,102,104,106,108,110,114,116,118,120,122,124,130,132,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,252,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,280,282,284,286,287,288,289,290,291,292,293,294,295,296,297,298,

In [101]:

del demodata['NAME']
del demodata[0]
del demodata[1]
del demodata['State'] #delete unneeded columns

demodata['CountyName']=demodata['CountyName'].astype(str) #change column type

demodata=demodata[~demodata['CountyName'].str.startswith('Kalawao')] #remove kalawao county from dataset

demodata2=demodata.append(demodata)
demodata3=demodata2.append(demodata)
demodata4=demodata3.append(demodata)
demodata5=demodata4.append(demodata)
demodata6=demodata5.append(demodata)
demodata7=demodata6.append(demodata)
demodata8=demodata7.append(demodata)
demodata9=demodata8.append(demodata)
demodata10=demodata9.append(demodata)
demodata11=demodata10.append(demodata)
demodata12=demodata11.append(demodata)
demodata13=demodata12.append(demodata)
demodata14=demodata13.append(demodata)
demodata15=demodata14.append(demodata)
demodata16=demodata15.append(demodata)
demodata17=demodata16.append(demodata)
demodata18=demodata17.append(demodata)
demodata19=demodata18.append(demodata)
demodata20=demodata19.append(demodata)
demodata21=demodata20.append(demodata)
demodata22=demodata21.append(demodata)
demodata23=demodata22.append(demodata)
demodata24=demodata23.append(demodata) #repeat dataset, convert from 1 yearly to 24 monthly obs

In [102]:
demodata=demodata24.sort_values('CountyName')
urvaxdata=urvaxdata.sort_values('CountyName') #sort datasets by alphabet

In [103]:
date_col=urvaxdata.filter(['Date'])
date_col=date_col.reset_index()
demodata=demodata.reset_index()
demodata=pd.concat([date_col['Date'], demodata], axis=1) #add date col to demo data so it can be merged 

In [104]:
demo_vax_ur=pd.merge(demodata, urvaxdata,  how='left', left_on=['Date','fips'], right_on = ['Date','fips'])
demo_vax_ur=demo_vax_ur.rename(columns={'DP05_0002PE': 'Percent_Male', 'DP05_0003PE': 'Percent_Female', 'DP05_0018E':'Median_Age','DP05_0037PE':'Percent_White','DP05_0038PE':'Percent_Black', 'DP05_0039PE':'Percent_AmerInd', 'DP05_0044PE':'Percent_Asian', 'DP05_0052PE':'Percent_PacI', 'DP05_0071PE':'Percent_HisLat'})
demo_vax_ur=demo_vax_ur.sort_values(['CountyName_x', 'Date'])
demo_vax_ur=demo_vax_ur[~demo_vax_ur['CountyName_x'].str.endswith('AK')] #merge and filter dataset

In [105]:
poli_data=pd.read_csv(path+'/2020_US_County_Level_Presidential_Results.csv') #open political affiliation data

poli_data['state_name']=poli_data['state_name'].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY','District of Columbia':'DC'})
#replace state name to match other dfs
poli_data['CountyName_x']=poli_data['county_name']+', '+poli_data['state_name'] #new column with County, ST to match format in other df

In [106]:
biden=[]

for i in poli_data['diff']:
    if i<0:
        biden.append(True)
    else:
        biden.append(False)
        
trump=[]

for j in poli_data['diff']:
    if j>0:
        trump.append(True)
    else:
        trump.append(False) #create binary variables for political affiliation

poli_data['biden']=biden
poli_data['trump']=trump #add binary columns to dataset 

In [107]:
poli_data=pd.get_dummies(data=poli_data, columns=['biden', 'trump']) #creates dummies

In [108]:
poli_data=poli_data[['county_fips','CountyName_x','biden_True','trump_True']] #filters to necessary columns only

poli_data=poli_data[~poli_data['CountyName_x'].str.endswith('AK')] #removes entries from alaska 
poli_data['county_fips']=poli_data['county_fips'].astype(str)

In [109]:
poli_data=poli_data.rename(columns={'county_fips': 'fips'}) #rename fips codes 

In [110]:
poli_data2=poli_data.append(poli_data)
poli_data3=poli_data2.append(poli_data)
poli_data4=poli_data3.append(poli_data)
poli_data5=poli_data4.append(poli_data)
poli_data6=poli_data5.append(poli_data)
poli_data7=poli_data6.append(poli_data)
poli_data8=poli_data7.append(poli_data)
poli_data9=poli_data8.append(poli_data)
poli_data10=poli_data9.append(poli_data)
poli_data11=poli_data10.append(poli_data)
poli_data12=poli_data11.append(poli_data)
poli_data13=poli_data12.append(poli_data)
poli_data14=poli_data13.append(poli_data)
poli_data15=poli_data14.append(poli_data)
poli_data16=poli_data15.append(poli_data)
poli_data17=poli_data16.append(poli_data)
poli_data18=poli_data17.append(poli_data)
poli_data19=poli_data18.append(poli_data)
poli_data20=poli_data19.append(poli_data)
poli_data21=poli_data20.append(poli_data)
poli_data22=poli_data21.append(poli_data)
poli_data23=poli_data22.append(poli_data)
poli_data24=poli_data23.append(poli_data)

poli_data=poli_data24.sort_values('CountyName_x') #convert yearly dataset to monthly for 2 years and sort 

In [111]:
poli_data=poli_data.reset_index()
demo_vax_ur=demo_vax_ur.reset_index()
poli_demo_ur_vax=pd.concat([poli_data,demo_vax_ur], axis=1)
#filter and combine datasets

In [112]:
poli_demo_ur_vax.loc[~poli_demo_ur_vax.index.duplicated(), :] #remove dupliate rows 

,index,fips,CountyName_x,biden_True,trump_True,level_0,Date,index,fips,Percent_Male,...,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_5PlusPop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,2326,45001,"Abbeville County, SC",0,1,1,2020-01-31,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2326,45001,"Abbeville County, SC",0,1,15,2020-02-29,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2326,45001,"Abbeville County, SC",0,1,21,2020-03-31,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2326,45001,"Abbeville County, SC",0,1,16,2020-04-30,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2326,45001,"Abbeville County, SC",0,1,13,2020-05-31,2317,45001,48.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74683,2437,46137,"Ziebach County, SD",1,0,75383,2021-08-31,2428,46137,49.3,...,5.0,NaN,5.0,5.0,6.0,2756.0,NaN,2332.0,1997.0,NaN
74684,2437,46137,"Ziebach County, SD",1,0,75376,2021-09-30,2428,46137,49.3,...,5.0,NaN,5.0,5.0,6.0,2756.0,NaN,2332.0,1997.0,NaN
74685,2437,46137,"Ziebach County, SD",1,0,75378,2021-10-31,2428,46137,49.3,...,5.0,NaN,5.0,6.0,7.0,2756.0,NaN,2332.0,1997.0,NaN
74686,2437,46137,"Ziebach County, SD",1,0,75381,2021-11-30,2428,46137,49.3,...,5.0,NaN,6.0,6.0,7.0,2756.0,NaN,2332.0,1997.0,NaN


In [113]:
poli_demo_ur_vax=poli_demo_ur_vax.loc[:, poli_demo_ur_vax.columns.isin(['fips','Date','CountyName_x','biden_True','trump_True','Percent_Male', 'Percent_Female',
       'Percent_White', 'Percent_Black', 'Percent_AmerInd', 'Percent_Asian',
       'Percent_PacI','Percent_HisLat','UR_rate','Completeness_pct','Series_Complete_18PlusPop_Pct','Booster_Doses_Vax_Pct','Booster_Doses_18Plus','Metro_status'])]

In [114]:
mask=pd.read_csv(path+'/mask-use-by-county.csv') #open mask use dataset

In [115]:
mask['COUNTYFP']=mask['COUNTYFP'].astype(str)
akfips=['2013','2016','2020','2050','2060','2068','2070','2090','2100','2105','2110','2122','2130','2150','2158','2164','2170','2180','2185','2188','2195','2198','2220','2230','2240','2261','2275','2282','2290']
mask=mask[mask.COUNTYFP.isin(akfips) == False]
#filter out alaska data

In [116]:
mask=mask[~mask['COUNTYFP'].str.endswith('15005')] #remove entry from dataset

In [117]:
mask2=mask.append(mask)
mask3=mask2.append(mask)
mask4=mask3.append(mask)
mask5=mask4.append(mask)
mask6=mask5.append(mask)
mask7=mask6.append(mask)
mask8=mask7.append(mask)
mask9=mask8.append(mask)
mask10=mask9.append(mask)
mask11=mask10.append(mask)
mask12=mask11.append(mask)
mask13=mask12.append(mask)
mask14=mask13.append(mask)
mask15=mask14.append(mask)
mask16=mask15.append(mask)
mask17=mask16.append(mask)
mask18=mask17.append(mask)
mask19=mask18.append(mask)
mask20=mask19.append(mask)
mask21=mask20.append(mask)
mask22=mask21.append(mask)
mask23=mask22.append(mask)
mask24=mask23.append(mask)

mask=mask24.sort_values('COUNTYFP')
#change yearly dataset to 24 months 

In [118]:
poli_demo_ur_vax=poli_demo_ur_vax.loc[:,~poli_demo_ur_vax.columns.duplicated()]
mask=mask.rename(columns={'COUNTYFP': 'fips'}) #change name of fips code 

In [119]:
poli_demo_ur_vax=poli_demo_ur_vax.sort_values('fips') #sort by fips code 

In [120]:
mask=mask.reset_index(drop=True)
poli_demo_ur_vax=poli_demo_ur_vax.reset_index(drop=True)
mask_poli_demo_ur_vax=pd.concat([mask,poli_demo_ur_vax], axis=1)
mask_poli_demo_ur_vax #filter and commbine datasets

,fips,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,fips,CountyName_x,biden_True,trump_True,...,Percent_AmerInd,Percent_Asian,Percent_PacI,Percent_HisLat,UR_rate,Completeness_pct,Series_Complete_18PlusPop_Pct,Booster_Doses_Vax_Pct,Booster_Doses_18Plus,Metro_status
0,10001,0.026,0.006,0.106,0.051,0.811,10001,"Kent County, DE",1,0,...,0.7,2.0,0.1,7.1,5.2,96.3,58.3,NaN,NaN,Metro
1,10001,0.026,0.006,0.106,0.051,0.811,10001,"Kent County, DE",1,0,...,0.7,2.0,0.1,7.1,4.5,96.2,60.3,NaN,NaN,Metro
2,10001,0.026,0.006,0.106,0.051,0.811,10001,"Kent County, DE",1,0,...,0.7,2.0,0.1,7.1,4.4,NaN,NaN,NaN,NaN,NaN
3,10001,0.026,0.006,0.106,0.051,0.811,10001,"Kent County, DE",1,0,...,0.7,2.0,0.1,7.1,5.6,NaN,NaN,NaN,NaN,NaN
4,10001,0.026,0.006,0.106,0.051,0.811,10001,"Kent County, DE",1,0,...,0.7,2.0,0.1,7.1,15.8,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74683,9015,0.022,0.041,0.082,0.137,0.718,9015,"Windham County, CT",0,1,...,0.6,1.4,0.0,11.8,4.8,NaN,NaN,NaN,NaN,NaN
74684,9015,0.022,0.041,0.082,0.137,0.718,9015,"Windham County, CT",0,1,...,0.6,1.4,0.0,11.8,4.8,NaN,NaN,NaN,NaN,NaN
74685,9015,0.022,0.041,0.082,0.137,0.718,9015,"Windham County, CT",0,1,...,0.6,1.4,0.0,11.8,4.5,98.3,69.7,37.6,26458.0,Metro
74686,9015,0.022,0.041,0.082,0.137,0.718,9015,"Windham County, CT",0,1,...,0.6,1.4,0.0,11.8,7.4,NaN,NaN,NaN,NaN,NaN


In [121]:
gdp=pd.read_csv(path+'/gdp2020_21.csv')

In [122]:
mask_poli_demo_ur_vax['Quarter']=mask_poli_demo_ur_vax['Date'].dt.to_period('Q') #convert dates to quarter

In [123]:
mask_poli_demo_ur_vax['State']=[x[-2:] for x in mask_poli_demo_ur_vax['CountyName_x']] #create column State

In [124]:
gdp['State']=gdp['GeoName'].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY','District of Columbia':'DC'})

In [125]:
gdp=gdp.melt(id_vars=["State", "GeoName","GeoFips"], 
        var_name="Quarter", 
        value_name="Q_GDP")

#change from wide to long 

,State,GeoName,GeoFips,Quarter,Q_GDP
0,AL,Alabama,1000,2020:Q1,202069.1
1,AK,Alaska,2000,2020:Q1,52876.1
2,AZ,Arizona,4000,2020:Q1,324209.9
3,AR,Arkansas,5000,2020:Q1,116945.8
4,CA,California,6000,2020:Q1,2739612.3
...,...,...,...,...,...
403,VA,Virginia,51000,2021:Q4,500941.3
404,WA,Washington,53000,2021:Q4,582097.2
405,WV,West Virginia,54000,2021:Q4,72854.1
406,WI,Wisconsin,55000,2021:Q4,306733.0


In [126]:
gdp=gdp.sort_values('State') #sort alphabetically by state

In [127]:
gdp['quarterstate']=gdp['Quarter'].astype(str)+gdp['State'].astype(str)
gdp['quarterstate']=gdp['quarterstate'].str.replace(':', '')
#create individual identifier for state and quarter 

,State,GeoName,GeoFips,Quarter,Q_GDP,quarterstate
1,AK,Alaska,2000,2020:Q1,52876.1,2020Q1AK
154,AK,Alaska,2000,2020:Q4,50733.8,2020Q4AK
205,AK,Alaska,2000,2021:Q1,49820.2,2021Q1AK
52,AK,Alaska,2000,2020:Q2,47820.3,2020Q2AK
103,AK,Alaska,2000,2020:Q3,49213.8,2020Q3AK
...,...,...,...,...,...,...
152,WY,Wyoming,56000,2020:Q3,36061.7,2020Q3WY
101,WY,Wyoming,56000,2020:Q2,34678.2,2020Q2WY
50,WY,Wyoming,56000,2020:Q1,38285.8,2020Q1WY
356,WY,Wyoming,56000,2021:Q3,36565.4,2021Q3WY


In [128]:
mask_poli_demo_ur_vax['Year']=[x[0:4] for x in mask_poli_demo_ur_vax['Date'].astype(str)]
mask_poli_demo_ur_vax['Quarter']=mask_poli_demo_ur_vax['Quarter'].astype(str)
mask_poli_demo_ur_vax['quarterstate']=mask_poli_demo_ur_vax['Quarter']+mask_poli_demo_ur_vax['State']
mask_poli_demo_ur_vax=mask_poli_demo_ur_vax.sort_values('State')

#filter and rename columns 

,fips,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,fips,CountyName_x,biden_True,trump_True,...,UR_rate,Completeness_pct,Series_Complete_18PlusPop_Pct,Booster_Doses_Vax_Pct,Booster_Doses_18Plus,Metro_status,Quarter,State,Year,quarterstate
1253,1099,0.112,0.041,0.182,0.288,0.378,1099,"Monroe County, AL",0,1,...,5.4,91.7,32.8,NaN,NaN,Non-metro,2021Q3,AL,2021,2021Q3AL
1127,1087,0.008,0.033,0.241,0.172,0.545,1087,"Macon County, AL",1,0,...,4.1,NaN,NaN,NaN,NaN,NaN,2020Q1,AL,2020,2020Q1AL
1126,1087,0.008,0.033,0.241,0.172,0.545,1087,"Macon County, AL",1,0,...,7.1,93.8,7.6,NaN,NaN,Non-metro,2021Q1,AL,2021,2021Q1AL
1125,1087,0.008,0.033,0.241,0.172,0.545,1087,"Macon County, AL",1,0,...,14.3,NaN,NaN,NaN,NaN,NaN,2020Q2,AL,2020,2020Q2AL
1124,1087,0.008,0.033,0.241,0.172,0.545,1087,"Macon County, AL",1,0,...,3.9,NaN,NaN,NaN,NaN,NaN,2020Q1,AL,2020,2020Q1AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71268,56021,0.143,0.127,0.100,0.221,0.409,56021,"Laramie County, WY",0,1,...,5.5,99.0,1.7,NaN,NaN,Metro,2021Q1,WY,2021,2021Q1WY
71269,56021,0.143,0.127,0.100,0.221,0.409,56021,"Laramie County, WY",0,1,...,4.1,0.0,0.0,NaN,NaN,Metro,2020Q4,WY,2020,2020Q4WY
71270,56021,0.143,0.127,0.100,0.221,0.409,56021,"Laramie County, WY",0,1,...,2.3,96.4,58.2,NaN,NaN,Metro,2021Q4,WY,2021,2021Q4WY
71261,56021,0.143,0.127,0.100,0.221,0.409,56021,"Laramie County, WY",0,1,...,6.7,NaN,NaN,NaN,NaN,NaN,2020Q2,WY,2020,2020Q2WY


In [129]:
gdp_mask_poli_demo_ur_vax=pd.merge(mask_poli_demo_ur_vax, gdp,  how='left', left_on=['quarterstate'], right_on = ['quarterstate'])

#merge datasets 

,fips,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,fips,CountyName_x,biden_True,trump_True,...,Metro_status,Quarter_x,State_x,Year,quarterstate,State_y,GeoName,GeoFips,Quarter_y,Q_GDP
0,1099,0.112,0.041,0.182,0.288,0.378,1099,"Monroe County, AL",0,1,...,Non-metro,2021Q3,AL,2021,2021Q3AL,AL,Alabama,1000,2021:Q3,204968.8
1,1087,0.008,0.033,0.241,0.172,0.545,1087,"Macon County, AL",1,0,...,NaN,2020Q1,AL,2020,2020Q1AL,AL,Alabama,1000,2020:Q1,202069.1
2,1087,0.008,0.033,0.241,0.172,0.545,1087,"Macon County, AL",1,0,...,Non-metro,2021Q1,AL,2021,2021Q1AL,AL,Alabama,1000,2021:Q1,202598.8
3,1087,0.008,0.033,0.241,0.172,0.545,1087,"Macon County, AL",1,0,...,NaN,2020Q2,AL,2020,2020Q2AL,AL,Alabama,1000,2020:Q2,183890.3
4,1087,0.008,0.033,0.241,0.172,0.545,1087,"Macon County, AL",1,0,...,NaN,2020Q1,AL,2020,2020Q1AL,AL,Alabama,1000,2020:Q1,202069.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74683,56021,0.143,0.127,0.100,0.221,0.409,56021,"Laramie County, WY",0,1,...,Metro,2021Q1,WY,2021,2021Q1WY,WY,Wyoming,56000,2021:Q1,36425.2
74684,56021,0.143,0.127,0.100,0.221,0.409,56021,"Laramie County, WY",0,1,...,Metro,2020Q4,WY,2020,2020Q4WY,WY,Wyoming,56000,2020:Q4,36000.9
74685,56021,0.143,0.127,0.100,0.221,0.409,56021,"Laramie County, WY",0,1,...,Metro,2021Q4,WY,2021,2021Q4WY,WY,Wyoming,56000,2021:Q4,36979.4
74686,56021,0.143,0.127,0.100,0.221,0.409,56021,"Laramie County, WY",0,1,...,NaN,2020Q2,WY,2020,2020Q2WY,WY,Wyoming,56000,2020:Q2,34678.2


In [130]:
gdp_mask_poli_demo_ur_vax=gdp_mask_poli_demo_ur_vax[['fips','CountyName_x','Date','Quarter_x','NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS','biden_True', 'trump_True','Percent_Male', 'Percent_Female', 'Percent_White',
       'Percent_Black', 'Percent_AmerInd', 'Percent_Asian', 'Percent_PacI','Percent_HisLat',
       'UR_rate', 'Completeness_pct', 'Series_Complete_18PlusPop_Pct','Q_GDP','Year']]
#filter dataset

In [131]:
gdp_mask_poli_demo_ur_vax=gdp_mask_poli_demo_ur_vax.sort_values(['CountyName_x', 'Date'])
#srot by date and alphabet

In [175]:
policy=pd.read_csv(path+'/State-Level_Vaccine_Mandates_-_All.csv') #open vaccinee mandate dataset

In [176]:
policy['date_signed']=[x[0:10] for x in policy['date_signed']]
policy['date_signed']=policy['date_signed'].astype(str)
policy=policy[~policy['date_signed'].str.endswith('2022')]
#filter and organize date columns 

In [177]:
policy['date_signed']=pd.to_datetime(policy['date_signed'], errors='coerce')
policy=policy.sort_values(['state', 'date_signed'])
#filter by date and state

In [178]:
policy=policy[~policy['state'].str.endswith('Samoa')]
policy=policy[~policy['state'].str.endswith('Rico')]
policy=policy[~policy['state'].str.endswith('Islands')]
policy=policy[~policy['state'].str.endswith('Guam')]
#remove territories from dataset

In [179]:
policy['date_signed']=policy['date_signed'].dt.month
policy = policy.drop_duplicates(
  subset = ['state', 'date_signed'],
  keep = 'first').reset_index(drop = True)
#convert date to date time, drop duplicates 

In [180]:
newcol= pd.MultiIndex.from_product([policy['state'].unique(), 
                                  np.arange(policy['date_signed'].min(), policy['date_signed'].max() + 1)],
                                  names=['state','date_signed'])

policy= policy.set_index(['state','date_signed']).reindex(newcol).reset_index()

#arrange by date and state

In [181]:
policy=policy[['state','date_signed']]
policy['policy_true']=[1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,0,1,1,1,1]
policy['Policies']=[4/16,9/16,10/16,10/16,11/16,14/16,1/16,5/16,6/16,8/16,9/16,11/16,0,4/16,12/16,12/16,12/16,12/16,0,1/16,2/16,3/16,3/16,3/16,0,3/16,4/16,4/16,4/16,6/16,0,2/16,4/16,6/16,9/16,9/16,0,5/16,10/16,10/16,10/16,10/16,0,1/16,1/16,1/16,1/16,1/16,0,1/16,2/16,2/16,2/16,2/16,0,6/16,6/16,6/16,6/16,8/16,0,2/16,4/16,4/16,4/16,4/16,0,1/16,1/16,1/16,1/16,1/16,1/16,1/16,2/16,2/16,2/16,2/16,0,5/16,5/16,6/16,6/16,6/16,2/16,6/16,6/16,7/16,7/16,8/16,2/16,6/16,9/16,10/16,10/16,10/16,2/16,2/16,2/16,2/16,3/16,3/16,0,6/16,8/16,8/16,8/16,8/16,0,2/16,2/16,2/16,2/16,2/16,0,1/16,2/16,2/16,2/16,2/16,0,1/16,1/16,1/16,1/16,1/16,0,7/16,11/16,11/16,15/16,15/16,0,0,1/16,1/16,1/16,1/16]
#mmanually add dummy for whrther policy is active 
#manually add ratio variable of number of policies active 

In [182]:
policy['date_signed']=policy['date_signed'].astype(str)
policy['Date']=policy['date_signed'].replace({'7':'2021-07-31','8':'2021-08-31','9':'2021-09-30','10':'2021-10-31','11':'2021-11-30','12':'2021-12-31'})
#convert month number to last day of each month to match other dfs 

In [183]:
policy['State']=policy['state'].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY','District of Columbia':'DC'})

In [186]:
policy=policy[['Date','State','policy_true','Policies']]
policy['Date']=pd.to_datetime(policy['Date'], errors='coerce')
#filter policy dataset to needed columns 

<ipython-input-186-04e26ab8b9bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  policy['Date']=pd.to_datetime(policy['Date'], errors='coerce')


In [187]:
gdp_mask_poli_demo_ur_vax['State']=[x[-2:] for x in gdp_mask_poli_demo_ur_vax['CountyName_x']]
#create new column called state based on last 2 digits of countyname_x column 

In [188]:
policy_diff=pd.merge(gdp_mask_poli_demo_ur_vax, policy, on=['Date','State'],how='left') #merge datasets into final

In [189]:
policy_diff['treat_true']=np.where(((policy_diff['State']=='CA') | (policy_diff['State']=='CO') | (policy_diff['State']=='CT') | (policy_diff['State']=='DE')|(policy_diff['State']=='DC')|(policy_diff['State']=='HI')|(policy_diff['State']=='IL')|(policy_diff['State']=='KY')|(policy_diff['State']=='ME')|(policy_diff['State']=='MD')|(policy_diff['State']=='MA')|(policy_diff['State']=='MN')|(policy_diff['State']=='NV')|(policy_diff['State']=='NJ')|(policy_diff['State']=='NM')|(policy_diff['State']=='NY')|(policy_diff['State']=='NC')|(policy_diff['State']=='OR')|(policy_diff['State']=='RI')|(policy_diff['State']=='VT')|(policy_diff['State']=='VA')|(policy_diff['State']=='WA')|(policy_diff['State']=='WI')), 1,0)

In [196]:
policy_diff.to_csv('/Users/caseycloutier/Desktop/Theses/Honors585 Paper Data/cleaned data/finaldata.csv')
#save df as final 